In [1]:
import pandas as pd 
from pandasql import sqldf

__Pipeline Helper Functions__

In [2]:
def get_all_routes_from_cluster(locations_filepath,clusters_filepath,clusterNo):
#### Phase 1: Generate dataset of all possible routes from a given cluster number

    stations = pd.read_csv(locations_filepath)
    clusters = pd.read_csv(clusters_filepath).drop("Unnamed: 0", axis = 1)
    
    all_stationID = stations["TERMINAL_NUMBER"].values.tolist()
    
    q = """SELECT Station_Number FROM clusters WHERE cluster == {} """.format(clusterNo)
    C_stationID = sqldf(q,locals())
    C_stationID = C_stationID["Station_Number"].values.tolist()
    
    C_routes = pd.DataFrame(columns = ["Start_ids","End_ids"])
    
    d = 0
    for i in C_stationID:
        d = d+1
        for j in all_stationID:
            new_row = {'Start_ids':i,'End_ids':j}
            C_routes = C_routes.append(new_row, ignore_index=True)
        print("Phase 1: {}% Complete".format(round(d/len(C_stationID),4)*100))
        
    clusterIDs = clusters
        
    q = """SELECT *
       FROM C_routes
       LEFT JOIN clusterIDs
       ON C_routes.Start_ids = clusterIDs.Station_Number;"""

    C_routes = sqldf(q,locals())

    C_routes = C_routes.rename(columns={"StationName": "Start_station", 
                          "cluster": "Start_cluster"})

    C_routes = C_routes.drop("Station_Number", axis = 1)

    q = """SELECT *
           FROM C_routes
           LEFT JOIN clusterIDs
           ON C_routes.End_ids = clusterIDs.Station_Number;"""

    C_routes = sqldf(q,locals())

    C_routes = C_routes.rename(columns={"StationName": "End_station", 
                          "cluster": "End_cluster"})

    C_routes = C_routes.drop("Station_Number", axis = 1)

    #Remove NAN Rows
    C_routes = C_routes.dropna()
    
    #Add Trip Key & re-arrange columns
    TRIP_KEY = list(range(len(C_routes)))
    C_routes["TRIP_KEY"] = TRIP_KEY
    C_routes = C_routes[["TRIP_KEY", "Start_ids", "End_ids","Start_station","Start_cluster","End_station","End_cluster"]]

    print("Phase 1: Complete")
    return(C_routes)


    
    

In [3]:
def munge_raw_trips(data):
    
    #### Change 'Start date' format into 'datetime' ####
    data['Start date'] = pd.to_datetime(data['Start date'])
    
    #### This chunk of code will turn the time of day into number of seconds since 12:00am of that day ####
    data['Date'] = data['Start date'].dt.date
    diff = pd.to_datetime(data['Start date']) - pd.to_datetime(data['Date'])
    data['Time'] = diff.dt.total_seconds()
    
    #### This chunk of code will extract month and weekday ####
    data['Month'] = pd.to_datetime(data['Date']).dt.month
    data['Weekday'] = pd.to_datetime(data['Date']).dt.weekday
    data['Weekday'] = data['Weekday'].replace([0, 1, 2, 3, 4, 5, 6],
                                              ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"])

    #### Feel free to drop the ’Start date" column now that you've extracted all the information ####
    data = data.drop('Start date', axis = 1)
    data = data.drop('End date', axis = 1)
    data = data.drop('Member type', axis = 1)
    data = data.drop('Start station', axis = 1)
    data = data.drop('End station', axis = 1)
    data = data.drop('Bike number', axis = 1)
    
    #### Rename cols for consistency ####
    data = data.rename(columns={"Start station number": "Start_id", "End station number": "End_id"})
    
    return(data)

In [4]:
def munge_and_merge(List_of_CSV_Files):
    #### Phase 2: Munge and Merge All CSV Files
    all_trips= pd.DataFrame(columns = ["Duration","Start_id","End_id","Date","Time","Month","Weekday"])
    for file in List_of_CSV_Files:
        raw_trips = pd.read_csv(file)
        trips = munge_raw_trips(raw_trips)
        all_trips = all_trips.append(trips)
    print("Phase 2: Complete")
    return(all_trips)
    

    

In [5]:
def Subset_Morning_trips(df):
#### Phase 3:Select only morning trips
    Morning_trips = df[df['Time'] <= 36000] 
    Morning_trips = df[df['Time'] >= 18000] 
    print("Phase 3: Complete")
    return(Morning_trips)

In [6]:
def day_level (test,routes_df):
#### Phase 4: Convert to the day level, will be a sparce df only includeing trips from cluster2
    date = sqldf("SELECT DISTINCT(Date) FROM test",locals())
    date = date.values.tolist()
    routes = routes_df
    Finaldf = pd.DataFrame(columns = ["TRIP_KEY","Start_ids","End_ids","Start_station",
                                  "Start_cluster","End_station","End_cluster","Date",
                                 "AVG_Duration","Month","Weekday","trips"]) 
    iter = 0
    for d in date:
        iter= iter +1
        days = str(d)
        day = days[1:-1]
        routes["Date"] = days[2:-2]

        q = """SELECT Start_id, End_id,  AVG(Duration) as AVG_Duration, Month, Weekday, count(*) as trips
                FROM test
                WHERE Date == {}
                GROUP BY Start_id, End_id""".format(day)

        day_trips = sqldf(q,locals())

        #Combine with the daily trip routs
        q = """SELECT * FROM routes
                LEFT JOIN day_trips
                ON routes.Start_ids = day_trips.Start_id
                AND routes.End_ids = day_trips.End_id"""

        daily_route_totals = sqldf(q,locals())
        daily_route_totals = daily_route_totals.drop(['Start_id', 'End_id'], axis=1)

        Finaldf = pd.concat([Finaldf, daily_route_totals])
        Finaldf["trips"].fillna(value=0, inplace=True)
        Finaldf["AVG_Duration"].fillna(value=0, inplace=True)
        Finaldf["Month"].fillna(value=99, inplace=True)
        Finaldf["Weekday"].fillna(value=99, inplace=True)
        
        print("Phase 4: {}% complete".format(round(iter/len(date),4)*100))
     
    print("Phase 4: Complete")
    return(Finaldf)



In [7]:
def add_days(data):
    #### Phase 5: Add in the missing data for trips that did not occur
    
    #### Change 'Start date' format into 'datetime' ####
    data["Date"] = pd.to_datetime(data["Date"])
    
    #### This chunk of code will extract month and weekday ####
    data['Month'] = pd.to_datetime(data['Date']).dt.month
    data['Weekday'] = pd.to_datetime(data['Date']).dt.weekday
    data['Weekday'] = data['Weekday'].replace([0, 1, 2, 3, 4, 5, 6],
                                              ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"])
    
    print("Phase 5: Complete")
    
    return(data)
    

## Pipeline Function
__Input:__

1) a list of the filepath for each month's trip data csv files downloaded from: https://s3.amazonaws.com/capitalbikeshare-data/index.html

2) the filepath for a the csv file downloaded from: https://opendata.dc.gov/datasets/capital-bike-share-locations/data?page=5

3) the filepath of a dataframe comtaining stationIds and clusterIDs

4) the cluster of interest: an integer of 0,1,2,3,4,5,6,7,8, or 9

__Output:__ a sparse dataframe containing all possible routes starting from all stations in the cluster of interest and the counts of the number of trips along that route on each day

In [65]:
def pipeline(List_of_CSV_Trip_Files,
             station_locations_filepath = "Capital_Bike_Share_Locations.csv",
             clusters_w_stationsIDs_filepath = "station_cluster.csv",
             cluster_of_interest = 8
             ):
    
    routes = get_all_routes_from_cluster(station_locations_filepath,clusters_w_stationsIDs_filepath,cluster_of_interest)
    
    a = munge_and_merge(List_of_CSV_Trip_Files) #Phase2
    b = Subset_Morning_trips(a) #phase 3
    c = day_level(b, routes) #phase 4
    d = add_days(c) #phase 5
    return(d)
    
    

In [66]:
CB_Trip_filepaths = ["MarchTripData.csv", "AprilTripData.csv"]
MarchApril_data = pipeline(CB_Trip_filepaths)

Phase 1: 2.86% Complete
Phase 1: 5.71% Complete
Phase 1: 8.57% Complete
Phase 1: 11.43% Complete
Phase 1: 14.29% Complete
Phase 1: 17.14% Complete
Phase 1: 20.0% Complete
Phase 1: 22.86% Complete
Phase 1: 25.71% Complete
Phase 1: 28.57% Complete
Phase 1: 31.430000000000003% Complete
Phase 1: 34.29% Complete
Phase 1: 37.14% Complete
Phase 1: 40.0% Complete
Phase 1: 42.86% Complete
Phase 1: 45.71% Complete
Phase 1: 48.57% Complete
Phase 1: 51.43% Complete
Phase 1: 54.290000000000006% Complete
Phase 1: 57.14% Complete
Phase 1: 60.0% Complete
Phase 1: 62.86000000000001% Complete
Phase 1: 65.71000000000001% Complete
Phase 1: 68.57% Complete
Phase 1: 71.43% Complete
Phase 1: 74.29% Complete
Phase 1: 77.14% Complete
Phase 1: 80.0% Complete
Phase 1: 82.86% Complete
Phase 1: 85.71% Complete
Phase 1: 88.57000000000001% Complete
Phase 1: 91.43% Complete
Phase 1: 94.28999999999999% Complete
Phase 1: 97.14% Complete
Phase 1: 100.0% Complete
Phase 1: Complete
Phase 2: Complete
Phase 3: Complete
Phas

In [67]:
#MarchApril_data.to_csv("trips_MarchApril.csv", index = False) 

,TRIP_KEY,Start_ids,End_ids,Start_station,Start_cluster,End_station,End_cluster,Date,AVG_Duration,Month,Weekday,trips
0,0,31253,31037,19th & K St NW,8,Ballston Metro / Stuart St & 9th St N,1,2020-02-01,0.0,2,Saturday,0.0
1,1,31253,31034,19th & K St NW,8,Fairfax Dr & N Randolph St,1,2020-02-01,0.0,2,Saturday,0.0
2,2,31253,31249,19th & K St NW,8,Jefferson Memorial,6,2020-02-01,0.0,2,Saturday,0.0
3,3,31253,31803,19th & K St NW,8,Good Hope Rd & 14th St SE,1,2020-02-01,0.0,2,Saturday,0.0
4,4,31253,31038,19th & K St NW,8,Glebe Rd & 11th St N,4,2020-02-01,0.0,2,Saturday,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
20820,20820,31646,32093,Maine Ave & 9th St SW,8,White Oak Transit Center,2,2020-03-31,0.0,3,Tuesday,0.0
20821,20821,31646,31926,Maine Ave & 9th St SW,8,Wilson Blvd. & N. Vermont St.,2,2020-03-31,0.0,3,Tuesday,0.0
20822,20822,31646,31826,Maine Ave & 9th St SW,8,Minnesota Ave & R St SE,5,2020-03-31,0.0,3,Tuesday,0.0
20823,20823,31646,32236,Maine Ave & 9th St SW,8,Vy Reston Heights,4,2020-03-31,0.0,3,Tuesday,0.0
